In [1]:
import java.util.HashMap;
import java.util.Map;

public class Light {
    boolean on;
    short red;
    short green;
    short blue;
    short effect;

    /*  ANSI effects
        n	Name	Note
        0	Reset or normal	All attributes off
        1	Bold or increased intensity	As with faint, the color change is a PC (SCO / CGA) invention.[38][better source needed]
        2	Faint, decreased intensity, or dim	May be implemented as a light font weight like bold.[39]
        3	Italic	Not widely supported. Sometimes treated as inverse or blink.[38]
        4	Underline	Style extensions exist for Kitty, VTE, mintty and iTerm2.[40][41]
        5	Slow blink	Sets blinking to less than 150 times per minute
        6	Rapid blink	MS-DOS ANSI.SYS, 150+ per minute; not widely supported
        7	Reverse video or invert	Swap foreground and background colors; inconsistent emulation[42]
        8	Conceal or hide	Not widely supported.
        9	Crossed-out, or strike	Characters legible but marked as if for deletion. Not supported in Terminal.app
     */
    private final Map<Short, String> EFFECT = new HashMap<>();
    {
        // Map<"separator", not_used>
        EFFECT.put((short) 0, "Normal");
        EFFECT.put((short) 1, "Bold");
        EFFECT.put((short) 2, "Faint");
        EFFECT.put((short) 3, "Italic");
        EFFECT.put((short) 4, "Underline");
        EFFECT.put((short) 5, "Slow Blink");
        EFFECT.put((short) 6, "Fast Blink");
        EFFECT.put((short) 7, "Reverse");
        EFFECT.put((short) 8, "Conceal");
        EFFECT.put((short) 9, "Crossed_out");
    }

    /* Assign random colors and effects */
    public Light() {
        int maxColor = 255;
        int effect = 9;
        this.red = (short) (Math.random()*(maxColor+1));
        this.green = (short) (Math.random()*(maxColor+1));
        this.blue = (short) (Math.random()*(maxColor+1));
        this.effect = (short) (Math.random()*(effect+1));
    }

    public String getEffectTitle() {
        return EFFECT.get(this.effect);
    }

    public String getRGB() {
        return ( "#" +
         String.format("%02X", this.red) +
         String.format("%02X", this.green) + 
         String.format("%02X", this.blue) 
         );
    }

    /* toString output as key/values */
    public String toString() {
        return( "{" + 
            "\"red\": " + red + "," +
            "\"green\": " +  green + "," + 
            "\"blue\": " + blue + "," +
            "\"effect\": " + "\"" + EFFECT.get(effect) + "\"" +
            "}" );
    }

    public boolean isOn() {
        return on;
    }

    public void setOn(boolean on) {
        this.on = on;
    }

    public short getRed() {
        return red;
    }

    public short getGreen() {
        return green;
    }

    public short getBlue() {
        return blue;
    }

    public short getEffect() {
        return effect;
    }

    static public void main(String[] args) {
        // create and display LightBoard
        Light light = new Light();
        System.out.println(light);  // use toString() method
    }
    

}
Light.main(null);

{"red": 140,"green": 133,"blue": 178,"effect": "Faint"}


In [2]:
public class LightBoard {
    private Light[][] lights;

    /* Initialize LightBoard and Lights */
    public LightBoard(int numRows, int numCols) {
        this.lights = new Light[numRows][numCols];
        // 2D array nested loops, used for initialization
        for (int row = 0; row < numRows; row++) {
            for (int col = 0; col < numCols; col++) {
                lights[row][col] = new Light();  // each cell needs to be constructed
            }
        }
    }

    /* Output is intended for API key/values */
    public String toString() { 
        String outString = "[";
        // 2D array nested loops, used for reference
        for (int row = 0; row < lights.length; row++) {
            for (int col = 0; col < lights[row].length; col++) {
                outString += 
                // data
                "{" + 
                "\"row\": " + row + "," +
                "\"column\": " + col + "," +
                "\"light\": " + lights[row][col] +   // extract toString data
                "}," ;
            }
        }
        // remove last comma, newline, add square bracket, reset color
        outString = outString.substring(0,outString.length() - 1) + "]";
		return outString;
    }

    /* Output is intended for Terminal, effects added to output */
    public String toTerminal() { 
        String outString = "[";
        // 2D array nested loops, used for reference
        for (int row = 0; row < lights.length; row++) {
            for (int col = 0; col < lights[row].length; col++) {
                outString += 
                // reset
                "\033[m" +
                
                // color
                "\033[38;2;" + 
                lights[row][col].getRed() + ";" +  // set color using getters
                lights[row][col].getGreen() + ";" +
                lights[row][col].getBlue() + ";" +
                lights[row][col].getEffect() + "m" +
                // data, extract custom getters
                "{" +
                "\"" + "RGB\": " + "\"" + lights[row][col].getRGB() + "\"" +
                "," +
                "\"" + "Effect\": " + "\"" + lights[row][col].getEffectTitle() + "\"" +
                "}," +
                // newline
                "\n" ;
            }
        }
        // remove last comma, newline, add square bracket, reset color
        outString = outString.substring(0,outString.length() - 2) + "\033[m" + "]";
		return outString;
    }

    /* Output is intended for Terminal, draws color palette */
    public String toColorPalette() {
        // block sizes
        final int ROWS = 5;
        final int COLS = 10;

        // Build large string for entire color palette
        String outString = "";
        // find each row
        for (int row = 0; row < lights.length; row++) {
            // repeat each row for block size
            for (int i = 0; i < ROWS; i++) {
                // find each column
                for (int col = 0; col < lights[row].length; col++) {
                    // repeat each column for block size
                    for (int j = 0; j < COLS; j++) {
                        // print single character, except at midpoint print color code
                        String c = (i == (int) (ROWS / 2) && j == (int) (COLS / 2) ) 
                            ? lights[row][col].getRGB()
                            : (j == (int) (COLS / 2))  // nested ternary
                            ? " ".repeat(lights[row][col].getRGB().length())
                            : " ";

                        outString += 
                        // reset
                        "\033[m" +
                        
                        // color
                        "\033[38;2;" + 
                        lights[row][col].getRed() + ";" +
                        lights[row][col].getGreen() + ";" +
                        lights[row][col].getBlue() + ";" +
                        "7m" +

                        // color code or blank character
                        c +

                        // reset
                        "\033[m";
                    }
                }
                outString += "\n";
            }
        }
        // remove last comma, newline, add square bracket, reset color
        outString += "\033[m";
		return outString;
    }
    
    static public void main(String[] args) {
        // create and display LightBoard
        LightBoard lightBoard = new LightBoard(5, 5);
        System.out.println(lightBoard);  // use toString() method
        System.out.println(lightBoard.toTerminal());
        System.out.println(lightBoard.toColorPalette());
    }
}

In [3]:
LightBoard lightBoard = new LightBoard(5, 5);
System.out.println(lightBoard);  // use toString() method

[{"row": 0,"column": 0,"light": {"red": 182,"green": 125,"blue": 162,"effect": "Crossed_out"}},{"row": 0,"column": 1,"light": {"red": 14,"green": 158,"blue": 120,"effect": "Normal"}},{"row": 0,"column": 2,"light": {"red": 71,"green": 3,"blue": 164,"effect": "Bold"}},{"row": 0,"column": 3,"light": {"red": 56,"green": 65,"blue": 156,"effect": "Conceal"}},{"row": 0,"column": 4,"light": {"red": 39,"green": 14,"blue": 53,"effect": "Slow Blink"}},{"row": 1,"column": 0,"light": {"red": 46,"green": 86,"blue": 65,"effect": "Conceal"}},{"row": 1,"column": 1,"light": {"red": 148,"green": 123,"blue": 148,"effect": "Slow Blink"}},{"row": 1,"column": 2,"light": {"red": 169,"green": 111,"blue": 221,"effect": "Normal"}},{"row": 1,"column": 3,"light": {"red": 240,"green": 47,"blue": 6,"effect": "Slow Blink"}},{"row": 1,"column": 4,"light": {"red": 11,"green": 152,"blue": 73,"effect": "Faint"}},{"row": 2,"column": 0,"light": {"red": 71,"green": 230,"blue": 145,"effect": "Fast Blink"}},{"row": 2,"column"

In [4]:
LightBoard lightBoard = new LightBoard(5, 5);
System.out.println(lightBoard.toTerminal());

[{"RGB": "#6C5C4F","Effect": "Conceal"},
{"RGB": "#619A7B","Effect": "Underline"},
{"RGB": "#04C173","Effect": "Bold"},
{"RGB": "#91AA16","Effect": "Reverse"},
{"RGB": "#13125B","Effect": "Normal"},
{"RGB": "#BC0EBC","Effect": "Fast Blink"},
{"RGB": "#6ECACB","Effect": "Reverse"},
{"RGB": "#D08C8F","Effect": "Bold"},
{"RGB": "#8E1061","Effect": "Slow Blink"},
{"RGB": "#8A822E","Effect": "Faint"},
{"RGB": "#541831","Effect": "Normal"},
{"RGB": "#3D0680","Effect": "Crossed_out"},
{"RGB": "#10E3EF","Effect": "Fast Blink"},
{"RGB": "#98C13D","Effect": "Italic"},
{"RGB": "#C13FA1","Effect": "Italic"},
{"RGB": "#6F925B","Effect": "Underline"},
{"RGB": "#BF1123","Effect": "Fast Blink"},
{"RGB": "#DD1ECD","Effect": "Fast Blink"},
{"RGB": "#3956FD","Effect": "Italic"},
{"RGB": "#55E019","Effect": "Underline"},
{"RGB": "#4A5E47","Effect": "Bold"},
{"RGB": "#6851CC","Effect": "Underline"},
{"RGB": "#7930A0","Effect": "Slow Blink"},
{"RGB": "#5A40C7","Effect": "Slow Blink"},
{"RGB": "#F42076","Eff

In [5]:
LightBoard lightBoard = new LightBoard(5, 5);
System.out.println(lightBoard.toColorPalette());

                                                                                
                                                                                
     #D483E2         #E2F125         #457776         #832BA9         #C113B6    
                                                                                
                                                                                
                                                                                
                                                                                
     #0F68E5         #7084FA         #90C5C6         #DF8150         #0CC2B6    
                                                                                
                                                                                
                                                                                
                                                                                
     #C1DF08         #D1322C